In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
include("../VARs/utils.jl")
includet("../RS/hmm_est.jl")

In [6]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12

macro_data = percentchange(data_source[:CPI, :INDPRO], :log)

start = Date(1955, 01,01)
end_d = Date(2024, 07, 28)



display("Macro data")
display(returns_summarystats(to(from(macro_data, start),end_d), freq))

display("Correlations")
display(cor(values(to(from(macro_data, start),end_d)))) 

"Macro data"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
CPI,0.0123,0.2148,0.0354,0.0596,-0.2321,0.0851,0.0035,0.5472,2.7577,2.8885,0.0348
INDPRO,0.0358,0.7653,0.0243,0.0788,-1.7045,-3.1785,-0.0232,0.3076,48.6631,0.6803,0.0294


nothing

"Correlations"

2×2 Matrix{Float64}:
  1.0       -0.011246
 -0.011246   1.0

In [7]:
assets_names = [:CPI, :INDPRO]
n_assets = length(assets_names)

2

In [10]:
n_regimes = 4
hmm = hmm_est(macro_data, n_regimes)
regime_summary(hmm[1], assets_names, 12)


Regime,CPI,INDPRO
1,0.047,0.027
2,0.015,-0.019
3,0.103,0.024
4,0.009,0.038


Regime,CPI,INDPRO
1,0.006,0.020
2,0.015,0.084
3,0.010,0.024
4,0.007,0.019


,CPI,INDPRO
CPI,1.000,0.066
INDPRO,0.066,1.000


,CPI,INDPRO
CPI,1.000,-0.094
INDPRO,-0.094,1.000


,CPI,INDPRO
CPI,1.000,-0.148
INDPRO,-0.148,1.000


,CPI,INDPRO
CPI,1.000,0.169
INDPRO,0.169,1.000


Regimes transition matrix


Regime,1,2,3,4
1,0.780,0.006,0.032,0.183
2,0.000,0.854,0.043,0.103
3,0.075,0.062,0.851,0.013
4,0.176,0.024,0.000,0.800


In [11]:
scenarios = simulate_hmm(hmm[1], n_assets, 25*freq, 10_000)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,CPI,INDPRO
1,0.0269,0.0058
5,0.0326,0.0205
10,0.0333,0.0229
25,0.0338,0.0243


,CPI,INDPRO
1,0.0225,0.0649
5,0.0122,0.0211
10,0.0087,0.0137
25,0.0056,0.0082


,CPI,INDPRO
1,0.9902,-0.3633
5,0.7913,-0.5996
10,0.6165,-0.5288
25,0.4245,-0.3612


,CPI,INDPRO
1,1.1194,0.8607
5,0.7902,1.0858
10,0.5052,0.7712
25,0.3544,0.4168


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0124,-0.0071,-0.0025,0.0117,0.0228,0.0373,0.073,0.0844,0.0951
5,0.0109,0.0137,0.0159,0.0239,0.0307,0.0397,0.0555,0.0611,0.0677
10,0.0167,0.0189,0.0208,0.0271,0.0323,0.0386,0.0492,0.0529,0.0574
25,0.0224,0.0239,0.0252,0.0298,0.0334,0.0373,0.0439,0.0461,0.0486


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1724,-0.1384,-0.1078,-0.0316,0.01,0.0476,0.1048,0.1262,0.1532
5,-0.0387,-0.027,-0.0175,0.0085,0.0227,0.0344,0.0512,0.0575,0.0648
10,-0.0142,-0.007,-0.0014,0.0149,0.024,0.0323,0.0433,0.0469,0.051
25,0.003,0.0066,0.0099,0.0191,0.0248,0.0299,0.0369,0.0391,0.0417


In [81]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.212073  0.312126  0.388067  0.48415  0.673097

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 26.0  48.0  70.0  104.0  209.03

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.138995  0.196629  0.246045  0.309159  0.45166

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 42.0  78.0  113.0  168.0  285.0